In [1]:
from scipy.sparse.linalg import splu, factorized
from scipy.linalg import lu_factor, lu_solve
import numpy as np
np.set_printoptions(precision=4) # Print few decimal places
np.set_printoptions(suppress=True) # Suppress scientific notation
import cvxpy as cp
import pandas as pd
from numpy.linalg import cholesky as llt
import matplotlib.pyplot as plt
import scipy.stats as stats
import time

In [2]:
"""Instance Generation"""
  

'Instance Generation'

In [77]:
# Generates a Quadratic problem data
def generate_data(n, m, mean, std, lower, upper, alpha):
  # M = np.random.normal(mean, std, (n, n))

  M = np.zeros((n,n))
  num_nonzero = int(n * n * 0.15)
  indices = np.random.choice(n * n, num_nonzero, replace=False)
  for i in indices:
    row = i // n
    col = i % n
    M[row, col] = np.random.normal(mean, std)

  P = M @ M.T + alpha * np.eye(n)
  

  # A = np.random.normal(mean, std, (m, n))

  A = np.zeros((m,n))
  num_nonzero = int(m * n * 0.15)
  indices = np.random.choice(m * n, num_nonzero, replace=False)
  for i in indices:
    row = i // n
    col = i % n
    A[row, col] = np.random.normal(mean, std)


  q = np.random.normal(mean, std, n)

  u = np.random.uniform(lower, upper, m)
  l = -np.random.uniform(lower, upper, m)

  return {
      'P' : P,
      'A' : A,
      'q' : q,
      'u' : u,
      'l' : l
  }
 
# Generate the problem instances 
def problem_instance(distribution, dimensions, num_instances=10):
  
  # Gaussian dist param
  mean, std = distribution['mean'], distribution['std']
  
  # Uniform dist param
  lower, upper = distribution['lower'], distribution['upper']
  
  # Prob Dimensions
  n, m = dimensions['n'], dimensions['m']
  alpha = dimensions['alpha']
  
  # Generating the problem instances
  prob_instances = {}

  for i in range(num_instances):
    key = f'set_{i+1}'
    prob_instances[key] = generate_data(n, m, mean, std, lower, upper, alpha)
    
  return prob_instances

In [130]:
# Calling function to generate data
distribution = {'mean': 0, 'std': 1, 'lower': 0, 'upper': 1}
dimensions = {'n': 100, 'm': 10*100, 'alpha': 10**-2}
num_instances = 5

# num_instances is by default 10 so no need to include it, but we can generate an
# arbitrary amount by sending in num_instances=5
problem_data = problem_instance(distribution, dimensions, num_instances)

In [64]:
"""OSQP Algorithms"""

'OSQP Algorithms'

In [65]:
# Projection step
def proj(v, l, u, version):
  z_next = np.empty(v.shape[0])
  if (version == 'box'):
    for i in range(len(v)):
      if (v[i] <= l[i]):
        z_next[i] = l[i]
      elif (l[i] <= v[i] and v[i] <= u[i]):
        z_next[i] = v[i]
      elif (v[i] >= u[i]):
        z_next[i] = u[i]
      else:
        print("error in projection")
  return z_next

In [66]:
# OSQP ADMM implementation
def OSQP_ADMM(problem, hyper_parameters, var, factor):
    
    # Epsilon parameters
    e_abs, e_rel = hyper_parameters['e_abs'], hyper_parameters['e_rel']
    e_pinf, e_dinf = hyper_parameters['e_pinf'], hyper_parameters['e_dinf']
    sigma, alpha, rho = hyper_parameters['sigma'], hyper_parameters['alpha'], hyper_parameters['rho']
    
    # Problem data
    P, q, A, u, l = problem['P'], problem['q'], problem['A'], problem['u'], problem['l']
    
    # Variables
    x, y, z = var['x'], var['y'], var['z']
    
    # Factorization
    lu = factor['lu']
    
    epsilon_prim = e_abs + e_rel * max(np.max(np.abs(A @ x)), np.max(np.abs(z)))
    epsilon_dual = e_abs + e_rel * max(np.max(np.abs(P @ x)), np.max(np.abs(A.T @ y)), np.max(np.abs(q)))
    
    r_prim = A @ x - z
    r_dual = P @ x + q + A.T @ y

    # Linear System
    left = sigma * x - q + A.T @ (rho * z - y)
    x_bar_next = lu.solve(left)

    z_bar_next = A @ x_bar_next

    x_next = alpha * x_bar_next + (1 - alpha) * x

    # Projection
    v = alpha * z_bar_next + (1 - alpha) * z + (1 / rho) * y
    z_next = proj(v, l, u, 'box')

    y_next = y + rho * (alpha * z_bar_next + (1 - alpha) * z - z_next)

    # Optimality termination criteria
    if (np.max(np.abs(r_prim)) <= epsilon_prim and np.max(np.abs(r_dual)) <= epsilon_dual):
        print("Optimal Solution Found")
        print("Optimal Value x_next", 0.5*x_next.T @ P @ x_next + q.T @ x_next)
        print("Optimal Value x", 0.5*x.T @ P @ x + q.T @ x)
        return x_bar_next, z_bar_next, x_next, z_next, y_next, True

    # Primal Infeasibility termination criteria
    # !!!!!!!!!!!!!
    # Fix this this is the wrong formula
    # !!!!!!!!!!!!!
    sigma_x = x_next - x
    sigma_z = z_next - z
    sigma_y = y_next - y

    criteria_1 = np.max(np.abs(A.T @ sigma_y)) <= e_pinf * np.max(np.abs(sigma_y))
    criteria_2 = u.T @ np.maximum(sigma_y, np.zeros(u.shape[0])) + l.T @ np.minimum(sigma_y, np.zeros(u.shape[0])) <= e_pinf * np.max(np.abs(sigma_y))


    if (criteria_1 and criteria_2):
        print("Primal Infeasible")
        return x_bar_next, z_bar_next, x_next, z_next, y_next, True

    # Dual Infeasibility termination criteria
    criteria_3 = np.max(np.abs(P @ sigma_x)) <= e_dinf * np.max(np.abs(sigma_x))
    criteria_4 = q.T @ sigma_x <= e_dinf * np.max(np.abs(sigma_x))
    criteria_5 = (-e_dinf * np.max(np.abs(sigma_x)) <= A @ sigma_x) & (A @ sigma_x <= e_dinf * np.max(np.abs(sigma_x)))
    
    if (criteria_3 and criteria_4):
        print("Dual Infeasible")
        return x_bar_next, z_bar_next, x_next, z_next, y_next, True

    return x_bar_next, z_bar_next, x_next, z_next, y_next, False

In [132]:
def v_ADMM(problem, hyper_parameters, var, factor, iteration, prev):
    
    # Epsilon parameters
    e_abs, e_rel = hyper_parameters['e_abs'], hyper_parameters['e_rel']
    e_pinf, e_dinf = hyper_parameters['e_pinf'], hyper_parameters['e_dinf']
    sigma, alpha, rho = hyper_parameters['sigma'], hyper_parameters['alpha'], hyper_parameters['rho']
    v, n = hyper_parameters['v'], hyper_parameters['n']
    
    # Problem data
    P, q, A, u, l = problem['P'], problem['q'], problem['A'], problem['u'], problem['l']
    
    # Variables
    x, y, z = var['x'], var['y'], var['z']
    
    # Previous Variables
    x_prev, z_prev, y_prev, gamma_init = prev['x'], prev['z'], prev['y'], prev['gamma']
    
    # Factorization
    lu = factor['lu']
    
    epsilon_prim = e_abs + e_rel * max(np.max(np.abs(A @ x)), np.max(np.abs(z)))
    epsilon_dual = e_abs + e_rel * max(np.max(np.abs(P @ x)), np.max(np.abs(A.T @ y)), np.max(np.abs(q)))
    
    r_prim = A @ x - z
    r_dual = P @ x + q + A.T @ y

    # Linear System
    left = sigma * x - q + A.T @ (rho * z - y)
    x_bar_next = lu.solve(left)

    z_bar_next = A @ x_bar_next

    x_next_ = alpha * x_bar_next + (1 - alpha) * x

    # Projection
    p = alpha * z_bar_next + (1 - alpha) * z + (1 / rho) * y
    z_next_ = proj(p, l, u, 'box')

    y_bar_next = y + rho * (alpha * z_bar_next + (1 - alpha) * z - z_next_)
    
    # Acceleration step
    meu_k = 1 + ((iteration - 1) * (2 * iteration - 3) * (2 * iteration + 2 * v - 1)) / ((iteration + 2 * v - 1) * 
                                                                                       (2 * iteration + 4 * v - 1) * 
                                                                                       (2 * iteration + 2 * v - 3))
    rho_k = (4 * (2 * iteration + 2 * v - 1) * (iteration + v - 1)) / ((iteration + 2 * v - 1) * (2 * iteration + 4 * v - 1))
    
    y_hat_k = meu_k * y_bar_next + (1 - meu_k) * y_prev + rho_k * (y_prev - y_bar_next)
    x_hat_k = meu_k * x_next_ + (1 - meu_k) * x_prev + rho_k * (x_prev - x_next_)
    z_hat_k = meu_k * z_next_ + (1 - meu_k) * z_prev + rho_k * (z_prev - z_next_)
    
    gamma = (1/rho) * np.linalg.norm(y_hat_k - y, 2) ** 2 + rho * np.linalg.norm(r_dual, 2) ** 2
    
    # Guard condition
    if gamma < (n ** (iteration+1)) * gamma_init:
        x_next = x_hat_k
        z_next = z_hat_k
        y_next = y_hat_k
    else:
        y_next = y_bar_next
        x_next = x_next_
        z_next = z_next_
        

    # Optimality termination criteria
    if (np.max(np.abs(r_prim)) <= epsilon_prim and np.max(np.abs(r_dual)) <= epsilon_dual):
        print("Optimal Solution Found")
        print("Optimal Value x_next", 0.5*x_next.T @ P @ x_next + q.T @ x_next)
        print("Optimal Value x", 0.5*x.T @ P @ x + q.T @ x)
        return x_bar_next, z_bar_next, x_next, z_next, y_next, y_bar_next, x_next_, z_next_, True

    # Primal Infeasibility termination criteria
    # !!!!!!!!!!!!!
    # Fix this this is the wrong formula
    # !!!!!!!!!!!!!
    sigma_x = x_next - x
    sigma_z = z_next - z
    sigma_y = y_next - y

    criteria_1 = np.max(np.abs(A.T @ sigma_y)) <= e_pinf * np.max(np.abs(sigma_y))
    criteria_2 = u.T @ np.maximum(sigma_y, np.zeros(u.shape[0])) + l.T @ np.minimum(sigma_y, np.zeros(u.shape[0])) <= e_pinf * np.max(np.abs(sigma_y))


    if (criteria_1 and criteria_2):
        print("Primal Infeasible")
        return x_bar_next, z_bar_next, x_next, z_next, y_next, y_bar_next, x_next_, z_next_, True

    # Dual Infeasibility termination criteria
    criteria_3 = np.max(np.abs(P @ sigma_x)) <= e_dinf * np.max(np.abs(sigma_x))
    criteria_4 = q.T @ sigma_x <= e_dinf * np.max(np.abs(sigma_x))
    criteria_5 = (-e_dinf * np.max(np.abs(sigma_x)) <= A @ sigma_x) & (A @ sigma_x <= e_dinf * np.max(np.abs(sigma_x)))
    
    if (criteria_3 and criteria_4):
        print("Dual Infeasible")
        return x_bar_next, z_bar_next, x_next, z_next, y_next,y_bar_next, x_next_, z_next_, True
    
    return x_bar_next, z_bar_next, x_next, z_next, y_next, y_bar_next, x_next_, z_next_, False

In [68]:
# Calling the ADMM recursively
# problem is P, q, A, u, l
# hyp_var is all epsilons, sigma, alpha and rho
# var is the starting point
# factor is is the lu factorization
def OSQP(problem, hyper_parameters, var, factor, version, max_iter=4000, v=0, n=0):
    fin = False
    
    x_bar_all = np.full((max_iter, var['x'].shape[0]), np.nan)
    x_all = np.full((max_iter, var['x'].shape[0]), np.nan)
    z_bar_all = np.full((max_iter, var['z'].shape[0]), np.nan)
    z_all = np.full((max_iter, var['z'].shape[0]), np.nan)
    y_all = np.full((max_iter, var['y'].shape[0]), np.nan)
    
    
    
    if version == 'v':
        x_ = np.zeros(var['x'].shape[0])
        z_ = np.zeros(var['z'].shape[0])
        y_bar = np.zeros(var['y'].shape[0])
        
        prev = {'y': y_bar, 'x': x_, 'z': z_, 'gamma': 1}
        hyper_parameters['v'] = v
        hyper_parameters['n'] = n
    
    if version == 'Original':
        for i in range(max_iter):
            x_bar, z_bar, x, z, y, fin = OSQP_ADMM(problem, hyper_parameters, var, factor)
            
            var = {'x': x, 'z': z, 'y': y}
            
            x_bar_all[i] = x_bar
            x_all[i] = x
            z_bar_all[i] = z_bar
            z_all[i] = z
            y_all[i] = y
            
            if fin:
                print("Iterations =", i)
                break
            
    elif version == 'v':
        for i in range(max_iter):
            x_bar, z_bar, x, z, y, y_bar, x_, z_, fin = v_ADMM(problem, hyper_parameters, var, factor, i, prev)
            
            var = {'x': x, 'z': z, 'y': y}
            prev = {'y': y_bar, 'x': x_, 'z': z_, 'gamma': 1}
            
            x_bar_all[i] = x_bar
            x_all[i] = x
            z_bar_all[i] = z_bar
            z_all[i] = z
            y_all[i] = y
            
            if fin:
                print("Iterations =", i)
                break
            
    return x_bar_all, z_bar_all, x_all, z_all, y_all, fin

In [146]:
# Setting the parameters
set_x = 'set_2'

P = problem_data[set_x]['P']
q = problem_data[set_x]['q']
A = problem_data[set_x]['A']
u = problem_data[set_x]['u']
l = problem_data[set_x]['l']

x = np.zeros(A.shape[1])
z = np.zeros(A.shape[0])
y = np.zeros(A.shape[0])

hyper_parameters = {'e_abs': 10**-3, 'e_rel': 10**-3, 'e_pinf': 10**-4, 'e_dinf': 10**-4,
                    'sigma': 10**-6, 'alpha': 1.6, 'rho': 0.1}

problem = {'P': P, 'q': q, 'A': A, 'u': u, 'l': l}

var = {'x': x, 'z': z, 'y': y}

matrix = P + hyper_parameters['sigma'] * np.eye(P.shape[0]) + hyper_parameters['rho'] * A.T @ A
lu = splu(matrix)

factor = {'lu': lu}

# Calling OSQP
x_bar, z_bar, x, z, y, fin = OSQP(problem, hyper_parameters, var, factor, version='Original')

v = 0.3
n = 0.999
# x_bar, z_bar, x, z, y, fin = OSQP(problem, hyper_parameters, var, factor, version='v', v=v, n=n)

C:\Users\baice\AppData\Local\Temp\ipykernel_4008\1308509867.py:22: SparseEfficiencyWarning: splu converted its input to CSC format
  lu = splu(matrix)


Optimal Solution Found
Optimal Value x_next -1.6410689924951547
Optimal Value x -1.641091049140874
Iterations = 500


In [147]:
# Checking against cvxpy

x_cvxpy = cp.Variable(A.shape[1])
objective = cp.Minimize(0.5 * cp.quad_form(x_cvxpy, P) + q.T @ x_cvxpy)
constraints = [A @ x_cvxpy <= u, l <= A @ x_cvxpy]
problem = cp.Problem(objective, constraints)
problem.solve()
print("CVXPY objective value", objective.value)

CVXPY objective value -1.6379151016432831
